# 1일차 종합실습 : 제조 공정간 불량 예측



## pulp-and-paper mill 공정
* 비즈니스 상황
    * 한 롤로 종이를 말다가 찢어지는 사고가 하루에 한번 이상 발생
    * 이때마다 공정 중단 및 수율 저하 등, 평균적으로 100백만원의 손실
    * 이를 사전에 감지하는 것은 굉장히 어려운 일입니다. 이런 사고를 5%만 감소시키더라도 회사 입장에서는 상당한 비용 절감 효과가 예상됩니다.

* Data
    * 행
        * 주어진 데이터에는 15일 동안 수집된 약 18,000개의 행의 시계열 데이터
        * 데이터는 2분 간격으로 측정.
    * 열 
        * y : normal – 0, abnormal - 1 (124건, 약 0.6%)
            * y 의 abnormal 데이터는 장애 발생 2~4분 전으로 시점에 대한 조정(shift)이 된(전처리 된) 데이터 입니다.
        * x1 - x61: 원자재, 부자재 및 공정 센서 값들로 구성됨.

* 장애 예방 조치
    * 장애가 예상된다면, 속도를 줄여 장애를 예방할 수 있습니다.
    * 단, 속도를 줄이면 생산성이 저하되므로, 1회당 평균 3만원의 손실이 발생됩니다.


![](https://keralakaumudi.com/web-news/en/2020/04/NMAN0141956/image/paper-mill.1.582102.jpg)

## 0.환경준비

### 1) 라이브러리 로딩

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from sklearn.metrics import *
from sklearn.model_selection import train_test_split, GridSearchCV

### 2) 데이터셋 불러오기

In [ ]:
# 공정 데이터 불러오기
path = "https://raw.githubusercontent.com/DA4BAM/dataset/master/processminer2.csv"
data = pd.read_csv(path)
data.head()

### 3) 필요 함수들 생성

#### ① precision, recall, f1 curve

> * sklearn에서는 precision, recall curve만 제공됩니다. 
* 그래서, f1 curve도 추가해서 구하고, plot을 그립니다.



In [ ]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

def prec_rec_f1_curve(y, score, pos = 1) :
    precision, recall, thresholds  = precision_recall_curve(y, score, pos_label=1)
    f1 = 2 / (1/precision + 1/recall)

    # f1, recall, precision curve
    plt.plot(thresholds, np.delete(precision, -1), label = 'precision')
    plt.plot(thresholds, np.delete(recall, -1), label = 'recall')
    plt.plot(thresholds, np.delete(f1, -1), label = 'f1')

    # f1를 최대화 해주는 threshold
    thre = round(thresholds[np.argmax(f1)],4)
    f1s = round(max(f1),4)
    plt.axvline(thre , color = 'darkred', linewidth = .7)
    plt.axhline( f1s, color = 'darkred', linewidth = .7)
    plt.text(thre, .5, thre, color = 'darkred')
    plt.text(min(thresholds), f1s, f1s, color = 'darkred')

    plt.xlabel('Anomaly Score')
    plt.ylabel('F1 Score')
    plt.legend()
    plt.grid()
    plt.show()

    return precision, recall, f1, thresholds

#### ② threshold로 잘랐을 때, 분류 평가 함수


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

def classification_report2(y, pred, thresholds):
    pred_temp = np.where(pred > thresholds , 1, 0)

    print('< confusion matrix >\n')
    print(confusion_matrix(y, pred_temp))
    print('\n' + '='*60 + '\n')

    print('< classification_report >\n')
    print(classification_report(y, pred_temp))

    return confusion_matrix(y, pred_temp)

## 1.데이터 탐색

In [ ]:
target = 'y'

* Target 변수의 class 비율을 확인해 봅시다.

In [ ]:
print(data[target].value_counts())
print(data[target].value_counts() / data.shape[0])


## 2.데이터 준비

### 2.2 추가변수

In [ ]:
for v in data.columns[2:] :
    var = v + '_diff'
    data[var] = data[v] - data[v].shift()

data.dropna(axis = 0, inplace = True)

### 2.1 x, y로 분할하기

* 불필요한 변수 제거

In [ ]:
data.drop('time', axis = 1, inplace = True)

In [ ]:
x = data.drop(target, axis = 1)
y = data.loc[:, target]

### 2.3 데이터 분할
* data ==> train : val = 7 : 3
* stratify=y : y의 class 비율을 유지하면서 분할하기

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = .3, random_state=2022, stratify=y)

In [ ]:
y_train.value_counts()/len(y_train)

In [ ]:
y_val.value_counts()/len(y_val)

In [ ]:
y_val.value_counts()

## 3.모델링 

* 알고리즘 선정 : 1번과 2번은 지도학습 알고리즘 중 원하는 것 2개 정도를 정해서 수행해 봅시다.
* 성능평가 : f1 score가 높은 모델/방식을 찾아 봅시다.

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import IsolationForest

### 1) resampling

In [ ]:
# down sampling


In [ ]:
# up sampling


In [ ]:
# SMOTE


* 모델링

### 2) class weight 조정


### 3) isolation forest


* training

* validation
    * Score 계산
    * predict로 예측

## 4.성능 비교

* 성능 평가 : F1 Score


In [ ]:
# resampling



In [ ]:
# class weights 조정



In [ ]:
# isolation forest

